In [2]:
%%capture
!pip install openai-whisper
!pip install flask
!pip install pyngrok
!pip install werkzeug

In [3]:
import whisper
import os
import tempfile
from flask import Flask, request, jsonify
import numpy as np
import io
import wave
from pyngrok import ngrok

In [4]:
app = Flask(__name__)

# Whisper モデルをロードします (tiny, base, small, medium, large から選択可能)
print("Whisper をロード中...")
model = whisper.load_model("base")  # "small", "medium", "large" など
print("モデルのロードが完了しました！")

Whisper をロード中...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 146MiB/s]


モデルのロードが完了しました！


In [5]:
@app.route('/v1/audio/transcriptions', methods=['POST'])
def transcribe():
  try:
    # ファイルがアップロードされているか確認
    if 'file' not in request.files:
      return jsonify({'error': 'ファイルが提供されていません'}), 400

    file = request.files['file']
    if file.filename == '':
      return jsonify({'error': 'ファイルが選択されていません'}), 400

    # 他のパラメータを取得
    model_name = request.form.get('model', 'whisper-1')
    language = request.form.get('language', None)
    prompt = request.form.get('prompt', None)
    temperature = float(request.form.get('temperature', 0.0))
    response_format = request.form.get('response_format', 'text')

    # アップロードされた音声ファイルを一時ファイルに保存
    with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_file:
      file.save(temp_file.name)
      temp_filename = temp_file.name

    try:
      # Whisperで文字起こしを実行
      options = {
        'temperature': temperature,
      }

      if language:
        options['language'] = language

      if prompt:
        options['initial_prompt'] = prompt

      result = model.transcribe(temp_filename, **options)

      # レスポンス形式に応じて結果を返す
      if response_format == 'text':
        return result['text']
      else:
        return jsonify(result)

    finally:
      # 一時ファイルを削除
      os.unlink(temp_filename)

  except Exception as e:
    return jsonify({'error': str(e)}), 500

In [6]:
@app.route('/health', methods=['GET'])
def health_check():
  return jsonify({'status': 'healthy', 'model': 'whisper'})

In [ ]:
if __name__ == '__main__':
  # ngrok 認証トークンを設定（ngrokのサイトから取得してください）
  ngrok.set_auth_token("ngrok_token")  # ご自身のngrokトークンに置き換えてください

  # ngrokトンネルを起動
  print("ngrokトンネルを起動中...")
  public_url = ngrok.connect(5000)
  print(f"API_URL: {public_url}")
  print(f"転写エンドポイント: {public_url}/v1/audio/transcriptions")
  print(f"ヘルスチェック: {public_url}/health")

  # Flaskアプリケーションを起動
  app.run(host='0.0.0.0', port=5000)

ngrokトンネルを起動中...
API_URL: NgrokTunnel: "https://0e2c-34-87-67-92.ngrok-free.app" -> "http://localhost:5000"
転写エンドポイント: NgrokTunnel: "https://0e2c-34-87-67-92.ngrok-free.app" -> "http://localhost:5000"/v1/audio/transcriptions
ヘルスチェック: NgrokTunnel: "https://0e2c-34-87-67-92.ngrok-free.app" -> "http://localhost:5000"/health
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
